In [3]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple, List
import logging
from dataclasses import dataclass
import time
from data_generator import TestConfiguration, create_test_instance

In [4]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [5]:
@dataclass
class DPState:
    """Class to represent state in the Dynamic Programming solution."""
    capacity: np.ndarray  # Capacity vector for each day
    time: int            # Current time period
    
    def __hash__(self):
        return hash((tuple(self.capacity), self.time))
    
    def __eq__(self, other):
        return np.array_equal(self.capacity, other.capacity) and self.time == other.time

class DynamicProgramming:
    """Implementation of the optimal Dynamic Programming solution."""
    
    def __init__(self, instance: Dict):
        self.instance = instance
        self.params = instance['parameters']
        self.booking_classes = instance['booking_classes']
        self.arrival_probs = instance['arrival_probabilities']
        self.epsilon = instance['reservation_price_params']
        self.value_function = {}  # State -> Value mapping
        
    def solve(self) -> float:
        """Solve the DP problem and return optimal expected revenue."""
        start_time = time.time()
        
        # Initialize terminal states
        for capacity in self._generate_capacity_vectors():
            terminal_state = DPState(capacity=capacity, time=self.params.T + 1)
            self.value_function[terminal_state] = 0
            
        # Backward induction
        for t in range(self.params.T, 0, -1):
            logger.info(f"Processing time period {t}")
            for capacity in self._generate_capacity_vectors():
                state = DPState(capacity=capacity, time=t)
                self.value_function[state] = self._compute_value_function(state)
                
        # Get initial state value
        initial_capacity = np.full(self.params.N, self.params.C)
        initial_state = DPState(capacity=initial_capacity, time=1)
        optimal_value = self.value_function[initial_state]
        
        solve_time = time.time() - start_time
        logger.info(f"DP solution completed in {solve_time:.2f} seconds")
        return optimal_value, solve_time
    
    def _compute_value_function(self, state: DPState) -> float:
        """Compute the value function for a given state."""
        if state in self.value_function:
            return self.value_function[state]
            
        current_probs = self.arrival_probs[state.time]
        max_value = 0
        
        # Try different prices
        for price in np.linspace(self.params.price_min, self.params.price_max, 20):
            value = 0
            
            # No arrival probability
            no_arrival_prob = 1 - sum(current_probs.values())
            if no_arrival_prob > 0:
                next_state = DPState(capacity=state.capacity.copy(), 
                                   time=state.time + 1)
                value += no_arrival_prob * self.value_function[next_state]
            
            # For each possible booking class arrival
            for (arrival, departure), arrival_prob in current_probs.items():
                if arrival_prob > 0:
                    # Compute acceptance probability
                    eps = self.epsilon[(arrival, departure)]
                    accept_prob = max(0, 1 - eps * price)
                    
                    # Check if we have capacity
                    has_capacity = True
                    for day in range(arrival - 1, departure):
                        if state.capacity[day] < 1:
                            has_capacity = False
                            break
                    
                    if has_capacity and accept_prob > 0:
                        # Accepted booking
                        next_capacity = state.capacity.copy()
                        for day in range(arrival - 1, departure):
                            next_capacity[day] -= 1
                        next_state = DPState(capacity=next_capacity, 
                                           time=state.time + 1)
                        revenue = (departure - arrival + 1) * price
                        value += arrival_prob * accept_prob * (
                            revenue + self.value_function[next_state]
                        )
                    
                    # Rejected booking (due to either capacity or price)
                    next_state = DPState(capacity=state.capacity.copy(), 
                                       time=state.time + 1)
                    value += arrival_prob * (1 - accept_prob) * self.value_function[next_state]
            
            max_value = max(max_value, value)
        
        return max_value
    
    def _generate_capacity_vectors(self) -> List[np.ndarray]:
        """Generate all possible capacity vectors for the given problem size."""
        capacities = []
        for i in range(self.params.C + 1):
            capacity = np.full(self.params.N, i)
            capacities.append(capacity)
        return capacities

class StochasticApproximation:
    """Implementation of the Stochastic Approximation Algorithm."""
    
    def __init__(self, instance: Dict, learning_rate: float = 0.1, 
                 num_iterations: int = 1000):
        self.instance = instance
        self.params = instance['parameters']
        self.booking_classes = instance['booking_classes']
        self.arrival_probs = instance['arrival_probabilities']
        self.epsilon = instance['reservation_price_params']
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.prices = self._initialize_prices()
        
    def _initialize_prices(self) -> Dict[int, np.ndarray]:
        """Initialize price vectors for each time period."""
        return {t: np.full(self.params.N, 
                         (self.params.price_min + self.params.price_max) / 2)
                for t in range(1, self.params.T + 1)}
    
    def solve(self) -> Tuple[float, float]:
        """Run the SAA algorithm and return the expected revenue."""
        start_time = time.time()
        
        for iteration in range(self.num_iterations):
            if iteration % 100 == 0:
                logger.info(f"SAA iteration {iteration}")
            
            # Generate sample path
            sample_path = self._generate_sample_path()
            
            # Forward pass
            revenue, gradients = self._forward_pass(sample_path)
            
            # Update prices using gradients
            self._update_prices(gradients)
        
        # Evaluate final solution
        final_revenue = self._evaluate_solution()
        solve_time = time.time() - start_time
        
        logger.info(f"SAA solution completed in {solve_time:.2f} seconds")
        return final_revenue, solve_time
    
    def _generate_sample_path(self) -> List[Tuple]:
        """Generate a sample path of customer arrivals and reservation prices."""
        path = []
        for t in range(1, self.params.T + 1):
            # Generate arrival
            probs = self.arrival_probs[t]
            classes = list(probs.keys())
            probabilities = list(probs.values())
            
            if np.random.random() < sum(probabilities):
                booking_class = np.random.choice(len(classes), p=probabilities/sum(probabilities))
                arrival, departure = classes[booking_class]
                
                # Generate reservation price
                eps = self.epsilon[(arrival, departure)]
                max_price = 1/eps  # Price where acceptance probability becomes 0
                reservation_price = np.random.uniform(self.params.price_min, max_price)
                
                path.append((t, arrival, departure, reservation_price))
            else:
                path.append((t, None, None, None))
        
        return path
    
    def _forward_pass(self, sample_path: List[Tuple]) -> Tuple[float, Dict]:
        """Perform forward pass through the sample path and compute gradients."""
        total_revenue = 0
        gradients = {t: np.zeros(self.params.N) for t in range(1, self.params.T + 1)}
        capacity = np.full(self.params.N, self.params.C)
        
        for t, arrival, departure, reservation_price in sample_path:
            if arrival is not None:
                # Check capacity
                has_capacity = True
                for day in range(arrival - 1, departure):
                    if capacity[day] < 1:
                        has_capacity = False
                        break
                
                if has_capacity:
                    # Calculate average price for the stay
                    stay_prices = self.prices[t][arrival-1:departure]
                    avg_price = np.mean(stay_prices)
                    
                    # Check if customer accepts price
                    if reservation_price >= avg_price:
                        # Accept booking
                        revenue = (departure - arrival + 1) * avg_price
                        total_revenue += revenue
                        
                        # Update capacity
                        for day in range(arrival - 1, departure):
                            capacity[day] -= 1
                        
                        # Compute price gradients
                        for day in range(arrival - 1, departure):
                            gradients[t][day] += 1  # Simplified gradient
        
        return total_revenue, gradients
    
    def _update_prices(self, gradients: Dict):
        """Update prices using computed gradients."""
        for t in range(1, self.params.T + 1):
            self.prices[t] += self.learning_rate * gradients[t]
            # Project prices to feasible range
            self.prices[t] = np.clip(self.prices[t], 
                                   self.params.price_min, 
                                   self.params.price_max)
    
    def _evaluate_solution(self, num_samples: int = 1000) -> float:
        """Evaluate the current solution using Monte Carlo simulation."""
        total_revenue = 0
        
        for _ in range(num_samples):
            sample_path = self._generate_sample_path()
            revenue, _ = self._forward_pass(sample_path)
            total_revenue += revenue
        
        return total_revenue / num_samples

In [6]:
def run_experiment1():
    """Run Experiment 1: Solution Quality Assessment."""
    logger.info("Starting Experiment 1: Solution Quality Assessment")
    
    # Create test instance
    config = TestConfiguration()
    test_params = config.get_config(
        test_type='minimal',
        market_condition='standard',
        discretization='coarse'
    )
    
    # Override parameters for small instance
    test_params.update({
        'T': 10,  # Small booking horizon
        'N': 5,   # Small service horizon
        'C': 5    # Small capacity
    })
    
    instance = create_test_instance(
        demand_scenario='base',
        market_condition='standard',
        test_configuration=test_params,
        seed=42
    )
    
    # Solve using Dynamic Programming
    dp = DynamicProgramming(instance)
    dp_revenue, dp_time = dp.solve()
    
    # Solve using Stochastic Approximation
    saa = StochasticApproximation(instance)
    saa_revenue, saa_time = saa.solve()
    
    # Calculate gap
    gap_percentage = ((dp_revenue - saa_revenue) / dp_revenue) * 100
    
    # Print results
    logger.info("\nExperiment 1 Results:")
    logger.info(f"Dynamic Programming Revenue: ${dp_revenue:.2f}")
    logger.info(f"SAA Revenue: ${saa_revenue:.2f}")
    logger.info(f"Optimality Gap: {gap_percentage:.2f}%")
    logger.info(f"DP Solution Time: {dp_time:.2f} seconds")
    logger.info(f"SAA Solution Time: {saa_time:.2f} seconds")
    
    return {
        'dp_revenue': dp_revenue,
        'saa_revenue': saa_revenue,
        'gap_percentage': gap_percentage,
        'dp_time': dp_time,
        'saa_time': saa_time
    }

In [7]:
if __name__ == "__main__":
    results = run_experiment1()

INFO:__main__:Starting Experiment 1: Solution Quality Assessment
INFO:data_generator:Generated 15 booking classes with max LOS = 7
INFO:data_generator:Initialized DataGenerator with 15 booking classes
INFO:data_generator:
Initial Price Generation Analysis for STANDARD Market
INFO:data_generator:Initialization Strategy: MARKET_BASED
INFO:data_generator:
Overall Price Statistics:
INFO:data_generator:Average Price: $231.96
INFO:data_generator:Median Price: $230.11
INFO:data_generator:Price Range: $183.87 - $335.37
INFO:data_generator:Standard Deviation: $36.41
INFO:data_generator:
Day-of-Week Price Analysis:
INFO:data_generator:Sunday: $290.68 (±$24.79)
INFO:data_generator:Monday: $212.81 (±$16.87)
INFO:data_generator:Tuesday: $210.19 (±$20.21)
INFO:data_generator:Wednesday: $209.91 (±$18.27)
INFO:data_generator:Thursday: $236.19 (±$14.00)
/Users/aeziz-local/anaconda3/envs/hotel-dynamic-pricing/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slic

UnboundLocalError: cannot access local variable 'accept_prob' where it is not associated with a value